In [0]:
import yt.wrapper as yt
import uuid
import os

In [1]:
yt.config["pickling"]["dynamic_libraries"]["enable_auto_collection"] = False
yt.config["pickling"]["ignore_system_modules"] = True
yt.config["pickling"]["safe_stream_mode"] = False

In [2]:
working_dir = f"//tmp/examples/tractorun-tiny-stories-finetune-{uuid.uuid4()}"
yt.create("map_node", working_dir, recursive=True)
print(working_dir)

//tmp/examples/tractorun-tiny-stories-finetune-875dc7db-a990-4a66-93db-38836f8dfa1b


In [3]:
from tractorun.toolbox import Toolbox
from tractorun.run import run
from tractorun.mesh import Mesh
from tractorun.resources import Resources
from tractorun.backend.generic import GenericBackend
from tractorun.backend.tractorch import Tractorch
from tractorun.stderr_reader import StderrMode

In [4]:
from yt import type_info

TALES_PER_JOB = 500

hf_token = os.environ.get("YT_SECURE_VAULT_HF_TOKEN", "")
assert hf_token is not None, "set HF token in kernel's secrets to use llama"

schema = yt.schema.TableSchema(strict=False)
schema.add_column("text", type_info.String)

datasets_path = f"{working_dir}/datasets"
yt.create("map_node", datasets_path)


MAX_TOKENS = 1000


def prepare_dataset(toolbox: Toolbox):
    from vllm import LLM, SamplingParams
    import os
    
    os.environ["HF_TOKEN"] = hf_token
    os.environ["CUDA_VISIBLE_DEVICES"] = str(toolbox.coordinator.get_process_index())

    self_index = toolbox.coordinator.get_self_index()
    table_path = f"{datasets_path}/dataset_{self_index}"

    llm = LLM(model="meta-llama/Llama-3.2-3B-Instruct", seed=self_index,)

    sampling_params = SamplingParams(
        temperature=0.9,
        top_p=0.85,
        max_tokens=MAX_TOKENS,
    )
    conversations = [
        [
            {
                "role": "system",
                "content": "You are a professional storyteller. Write the story in one paragraph, without line breaks. A user will now ask you to tell a fairy tale, and you must create a story featuring Tracto.ai. Tracto.ai the a ai-startup that provides infrastructure for machine learning and big data processing."
            },
            {
                "role": "user",
                "content": f"Write the {index}th fairy tail about some animal please.",
            },
        ] for index in range(TALES_PER_JOB)
    ]

    results = llm.chat(
        messages=conversations,
        sampling_params=sampling_params,
    )

    tales = ({"text": result.outputs[0].text} for result in results)

    toolbox.yt_client.create("table", table_path, attributes={"schema": schema.to_yson_type()}, force=True)
    toolbox.yt_client.write_table(table_path, tales)


run(
    prepare_dataset,
    backend=GenericBackend(),
    proxy_stderr_mode=StderrMode.primary,
    yt_path=f"{working_dir}/tractorun_inference",
    mesh=Mesh(node_count=2, gpu_per_process=1, process_per_node=8, pool="fifo", pool_trees=["gpu_h200"]),
    resources=Resources(
        cpu_limit=64,
        memory_limit=322122547200,
    ),
)

dataset_parts = [f"{datasets_path}/dataset_{i}" for i in range(2 * 8)]
dataset_path = f"{datasets_path}/dataset"

yt.run_merge(
    dataset_parts,
    dataset_path,
)

2025-02-06 18:51:15,370	WARNING	Cannot locate file of the module (__name__: torch.ops, __file__: _ops.py)


2025-02-06 18:51:15,372	WARNING	Cannot locate file of the module (__name__: torch.classes, __file__: _classes.py)


2025-02-06 18:51:18,007	INFO	Operation started: https://playground.yt.nebius.yt/playground/operations/cd0e06f6-42ee0531-270703e8-1b2ce6c0/details


2025-02-06 18:51:18,031	INFO	( 0 min) operation cd0e06f6-42ee0531-270703e8-1b2ce6c0 starting


2025-02-06 18:51:18,561	INFO	( 0 min) operation cd0e06f6-42ee0531-270703e8-1b2ce6c0 initializing


...

2025-02-06 18:54:44,646	INFO	( 0 min) operation f598913-b2269a5e-270703e8-c888c333 starting


2025-02-06 18:54:45,177	INFO	( 0 min) operation f598913-b2269a5e-270703e8-c888c333 initializing


2025-02-06 18:54:45,733	INFO	( 0 min) Unrecognized spec: {'enable_partitioned_data_balancing': false}


2025-02-06 18:54:45,734	INFO	( 0 min) operation f598913-b2269a5e-270703e8-c888c333 completing


2025-02-06 18:54:46,262	INFO	( 0 min) operation f598913-b2269a5e-270703e8-c888c333 completed


In [5]:
from tractorun.backend.tractorch import YtDataset
from tractorun.backend.tractorch.serializer import TensorSerializer

from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig


class YTTransform:
    def __init__(self, tokenizer: AutoTokenizer):
        self._tokenizer = tokenizer

    def __call__(self, columns: list[str], row: dict) -> tuple:
        assert columns == ["text"]
        input_ids = self._tokenizer(yt.yson.get_bytes(row["text"]).decode(), padding="max_length", max_length=MAX_TOKENS)["input_ids"]
        return {
            "input_ids": input_ids,
        }


def get_dataset(
    path: str,
    tokenizer: AutoTokenizer,
    yt_client: yt.YtClient,
) -> tuple[YtDataset, YtDataset]:
    start = 0
    end = yt_client.get(path + "/@row_count")

    train_end = int(end * 0.8)
    eval_start = train_end + 1

    train_dataset = YtDataset(path=path, yt_client=yt_client, transform=YTTransform(tokenizer), start=start, end=train_end, columns=["text"])
    eval_dataset = YtDataset(path=path, yt_client=yt_client, transform=YTTransform(tokenizer), start=eval_start, end=end, columns=["text"])
    return train_dataset, eval_dataset

In [6]:
from transformers import (
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    AutoModelForCausalLM,
    GenerationConfig,
)
from transformers.trainer_pt_utils import AcceleratorConfig


def training(toolbox: Toolbox):
    model = AutoModelForCausalLM.from_pretrained(
        "roneneldan/TinyStories-3M",
        trust_remote_code=True,
        use_cache = False,
    )
    tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
    tokenizer.pad_token = tokenizer.eos_token
    data_collator = DataCollatorForLanguageModeling(tokenizer, pad_to_multiple_of=2, mlm=False)
    train_dataset, eval_dataset = get_dataset(
        path=dataset_path,
        tokenizer=tokenizer,
        yt_client=toolbox.yt_client,
    )
    args = TrainingArguments(
        output_dir="/tmp/results",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        eval_on_start=True,
        eval_strategy="epoch",
        num_train_epochs=8,
        weight_decay=0.1,
        lr_scheduler_type="constant",
        learning_rate=5e-5,
        save_steps=0.0,  # don't save checkpoints
        logging_dir=None,
        logging_strategy="epoch",
        fp16=True,
        push_to_hub=False,
        batch_eval_metrics=False,
        accelerator_config=AcceleratorConfig(
            split_batches=True,
            dispatch_batches=True,
        ),
    )
    args = args.set_dataloader(train_batch_size=16, drop_last=True)
    trainer = Trainer(
        model=model,
        processing_class=tokenizer,
        args=args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
    )
    trainer.train()
    if toolbox.coordinator.is_primary():
        toolbox.save_model(TensorSerializer().serialize(trainer.model))


run(
    training,
    backend=Tractorch(),
    yt_path=f"{working_dir}/tractorun_training",
    mesh=Mesh(node_count=1, gpu_per_process=1, process_per_node=8, pool="fifo", pool_trees=["gpu_h200"]),
    resources=Resources(
        cpu_limit=64,
        memory_limit=322122547200,
    ),
)

2025-02-06 18:54:49,190	WARNING	Cannot locate file of the module (__name__: torch.ops, __file__: _ops.py)


2025-02-06 18:54:49,192	WARNING	Cannot locate file of the module (__name__: torch.classes, __file__: _classes.py)


2025-02-06 18:54:53,428	INFO	Operation started: https://playground.yt.nebius.yt/playground/operations/eb6a1d17-8cc5bc72-270703e8-62e5f601/details


2025-02-06 18:54:53,451	INFO	( 0 min) operation eb6a1d17-8cc5bc72-270703e8-62e5f601 starting


2025-02-06 18:54:53,978	INFO	( 0 min) operation eb6a1d17-8cc5bc72-270703e8-62e5f601 initializing


...

2025-02-06 18:54:55,610	INFO	( 0 min) operation eb6a1d17-8cc5bc72-270703e8-62e5f601: running=0     completed=0     pending=1     failed=0     aborted=0     lost=0     total=1     blocked=0    


2025-02-06 18:54:57,806	INFO	( 0 min) operation eb6a1d17-8cc5bc72-270703e8-62e5f601: running=1     completed=0     pending=0     failed=0     aborted=0     lost=0     total=1     blocked=0    


2025-02-06 18:58:30,075	INFO	( 3 min) operation eb6a1d17-8cc5bc72-270703e8-62e5f601 completing


2025-02-06 18:58:30,603	INFO	( 3 min) operation eb6a1d17-8cc5bc72-270703e8-62e5f601 completed


RunInfo(operation_spec={'description': {'notebook_path': '//home/chiffa/tractorun_example_1/notebook'}, 'started_by': {'hostname': 'end-a100-0.exec-nodes-a100.tundra.svc.testy.k8s.nebius.yt', 'pid': 3421, 'command': ['/slot/sandbox/jlab/site-packages/ipykernel_launcher.py', '-f', '/slot/sandbox/.local/share/jupyter/runtime/kernel-017cbf55-741c-494d-bf7a-4483604e8ce7.json'], 'wrapper_version': '0.13.22', 'python_version': '3.12.8', 'user': 'root', 'platform': 'Debian GNU/Linux 12 (bookworm)'}, 'fail_on_job_restart': True, 'is_gang': True, 'annotations': {'is_tractorun': True}, 'tasks': {'task': {'command': 'python3 _py_runner.py wrapped.pickle config_dump _modules_info _main_module.py _main_module PY_SOURCE', 'job_count': 1, 'gpu_limit': 8, 'port_count': 8, 'cpu_limit': 64, 'memory_limit': 322133234892, 'docker_image': 'cr.eu-north1.nebius.cloud/e00faee7vas5hpsh3s/chiffa/example:v1', 'file_paths': [{'value': '//tmp/yt_wrapper/file_storage/new_cache/dd/673cbb3cb215bad8d8d26e236296b8dd', 

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch
import io

incarnation = sorted(yt.list(f"{working_dir}/tractorun_training/models"), key=lambda x: int(x), reverse=True)[0]

raw_model = io.BytesIO(yt.read_file(f"{working_dir}/tractorun_training/models/{incarnation}").read())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load(raw_model).to(device)

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
tokenizer.pad_token = tokenizer.eos_token

/slot/sandbox/ipykernel_3421/2603817430.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(raw_model).to(device)


In [8]:
prompt = f"Long time ago"
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
output = model.generate(input_ids, max_length = 200, num_beams=1, temperature=0.7, do_sample=True)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

print("\n\n", output_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




 Long time ago a cunning fox named Kaito roamed with a thirst for adventure. One day, while exploring the forest, Kaito stumbled upon a hidden clearing where Tracto.ai, a mystical startup, had set up its headquarters. The CEO, a wise and kind fox named Nova, welcomed Kaito with open arms. The fox, with a curious mind and a mind full of wonder, sprang into action, with the help of Tracto.ai's magical algorithms. As they embarked on a quest to optimize the forest's systems, conjured a new system that illuminated the forest's growth. With the help of Tracto.ai's powerful infrastructure, the AI's agility, Kaito and the fox defeated the sly fox of bias and the treacherous landscape of data, and the fox's paws witherred, helping the team optimize their innovative solutions. And from that day on, Kaito and the Tracto.ai team worked tirelessly to develop a new era of
